In [1]:
# 导入包
from gurobipy import *
import pandas as pd

# 建立模型
m = Model(name='LogisticsExperiment-2')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-11


In [2]:
# 定义参数
# 房间数量
n = 8

# 部门/房间索引
set_N = range(1, n + 1)

# 各房间之间的运输距离
distance = pd.read_csv("distancez.csv",usecols=[i for i in set_N], dtype=int)
distance.index = [i for i in set_N]
distance.columns = [j for j in set_N]

# 各房间之间的流量
flow = pd.read_csv("flow.csv",usecols=[i for i in set_N], dtype=int)
flow.index = [i for i in set_N]
flow.columns = [j for j in set_N]

distance

,1,2,3,4,5,6,7,8
1,0,10,10,10,20,20,30,30
2,10,0,10,10,20,20,30,30
3,10,10,0,10,10,10,20,20
4,10,10,10,0,10,10,20,20
5,20,20,10,10,0,10,10,10
6,20,30,10,10,10,0,10,10
7,30,30,20,20,10,10,0,10
8,30,30,30,30,10,10,10,0


In [3]:
# 定义变量
x_vars = pd.DataFrame({j : pd.Series(m.addVar(vtype=GRB.BINARY,name="x_{0}_{1}".format(i,j)) 
                                     for i in set_N) 
                       for j in set_N})
x_vars.index = [i for i in set_N]

In [4]:
# 定义约束条件
columns_constrains = {j : 
                      m.addConstr(
                          sum(x_vars.loc[i,j] for i in set_N) == 1,
                          name="row_constrain_{0}".format(j))
                      for j in set_N}

rows_constraints = {i : 
                    m.addConstr(
                        sum(x_vars.loc[i,j] for j in set_N) == 1,
                        name="rows_constraint_{0}".format(i))
                    for i in set_N}

In [5]:
# 定义目标函数
objective = m.setObjective(sum(flow.loc[i, k] * distance.loc[j, l] * x_vars.loc[i, j] * x_vars.loc[k, l]
                               for l in set_N
                               for k in set_N
                               for j in set_N
                               for i in set_N),
                           GRB.MINIMIZE)

In [6]:
# 模型求解及结果输出
s = m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16 rows, 64 columns and 128 nonzeros
Model fingerprint: 0xf4b8b38c
Model has 616 quadratic objective terms
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+02, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 17100.000000
Presolve time: 0.00s
Presolved: 56 rows, 104 columns, 824 nonzeros
Variable types: 0 continuous, 104 integer (64 binary)

Root relaxation: objective 0.000000e+00, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   13 17100.0000    0.00000   100%     -    0s
H  

In [7]:
# 输出结果
for i in set_N:
    x_vars.loc[i] = x_vars.loc[i].apply(lambda item: int(item.x)) 

x_vars.to_csv("solution_x.csv")